# ODTC Door Command Test

This notebook demonstrates connecting to an ODTC device and testing the open/close door commands.

## Setup

Before running, ensure:
1. The ODTC device is powered on and connected to the network
2. You know the IP address of the ODTC device
3. Your computer is on the same network as the ODTC
4. The ODTC device is accessible (not locked by another client)

# 1. Libraries and configuration

In [1]:
# Import required modules
import logging

from pylabrobot.thermocycling.inheco import InhecoODTC, ODTCBackend

# Configure logging to show debug information
logging.basicConfig(
    level=logging.DEBUG,  # Set to DEBUG for more verbose output
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%H:%M:%S'
)

# Get logger for ODTC backend to see status debugging
odtc_logger = logging.getLogger('pylabrobot.thermocycling.inheco.odtc_backend')
odtc_logger.setLevel(logging.DEBUG)  # Set to DEBUG for more verbose output

sila_logger = logging.getLogger('pylabrobot.thermocycling.inheco.odtc_sila_interface')
sila_logger.setLevel(logging.DEBUG)  # Set to DEBUG for more verbose output

print("Logging configured. Debug information will be displayed.")

Logging configured. Debug information will be displayed.


In [2]:
# Configuration
backend = ODTCBackend(odtc_ip="192.168.1.50", logger=odtc_logger)
tc = InhecoODTC(name="odtc_test", backend=backend, model="96")
print("Backend and thermocycler objects created.")
print(f"Backend logger level: {backend.logger.level} ({logging.getLevelName(backend.logger.level)})")

Backend and thermocycler objects created.
Backend logger level: 10 (DEBUG)


# 2. Connection and method introspection

In [3]:
print("Starting device setup...")
print("=" * 60)

await tc.setup()

print("=" * 60)
print("✓ Connected and initialized successfully!")

Starting device setup...


2026-01-26 22:08:28,661 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - Device reset (unlocked)
2026-01-26 22:08:28,859 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - GetStatus returned raw state: 'standby' (type: str)
2026-01-26 22:08:28,860 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - Device is in standby state, calling Initialize...
2026-01-26 22:08:30,396 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - Device successfully initialized and is in idle state


✓ Connected and initialized successfully!


In [4]:
# List all methods and premethods stored on the device
# Note: Requires device to be in Idle state (after setup/initialize)
# Returns unified list of all method names (both methods and premethods)
method_names = await tc.list_methods()

print(f"Methods and PreMethods ({len(method_names)}):")
if method_names:
    for name in method_names:
        print(f"  - {name}")
else:
    print("  (none)")

# You can also get a specific method by name:
# method = await tc.get_method("PCR_30cycles")

Methods and PreMethods (77):
  - M18_Abnahmetest
  - M23_LEAK
  - M24_LEAKCYCLE
  - M22_A-RUNIN
  - M22_B-RUNIN
  - M30_PCULOAD
  - M33_Abnahmetest
  - M34_Dauertest
  - M35_VCLE
  - M36_Verifikation
  - M37_BGI-Kon
  - M39_RMATEST
  - M18_Abnahmetest_384
  - M23_LEAK_384
  - M22_A-RUNIN_384
  - M22_B-RUNIN_384
  - M30_PCULOAD_384
  - M33_Abnahmetest_384
  - M34_Dauertest_384
  - M35_VCLE_384
  - M36_Verifikation_384
  - M37_BGI-Kon_384
  - M39_RMATEST_384
  - M120_OVT
  - M320_OVT
  - M121_OVT
  - M321_OVT
  - M123_OVT
  - M323_OVT
  - M124_OVT
  - M324_OVT
  - M125_OVT
  - M325_OVT
  - PMA cycle
  - Test
  - DC4_ProK_digestion
  - DC4_3Prime_Ligation
  - DC4_ProK_digestion_1_test
  - DC4_3Prime_Ligation_test
  - DC4_5Prime_Ligation_test
  - DC4_USER_Ligation_test
  - DC4_5Prime_Ligation
  - DC4_USER_Ligation
  - DC4_ProK_digestion_37
  - DC4_ProK_digestion_60
  - DC4_3Prime_Ligation_Open_Close
  - DC4_3Prime_Ligation_37
  - DC4_5Prime_Ligation_37
  - DC4_USER_Ligation_37
  - Digestio

# 3. Commands

### Non-Blocking Door Operations

The door commands support non-blocking execution using the `wait=False` parameter. This returns a `CommandExecution` handle that you can await later, allowing you to do other work while the door operation completes. 

Note: Just set wait=True to treat these as blocking and run them as normal commands

In [5]:
# Non-blocking door close
print("Starting door close (non-blocking)...")
door_closing = await tc.close_door(wait=False)
print(f"✓ Door close command started! Request ID: {door_closing.request_id}")

Starting door close (non-blocking)...
✓ Door close command started! Request ID: 1351407823


# 5. Temperature Control

In [ ]:
# Set mount temperature to 37°C
# This creates a minimal protocol and uploads/runs it to the scratch file
# post_heating=True keeps temperatures held after method completes
print("Waiting for door to close...")
await door_closing.wait()
print("✓ Door close completed!")

print("Setting mount temperature to 37°C...")
execution = await tc.set_mount_temperature(
    37.0,
    wait=False,
    debug_xml=True,  # Enable XML logging (shows in console if DEBUG logging is on)
    xml_output_path="debug_set_mount_temp.xml"  # Save XML to file
)

if await tc.get_status() == "busy":
    print(f"✓ Method started! Request ID: {execution.request_id}")
    print(f"Current temperatures:\n{await tc.read_temperatures()}")
    print("Check debug_set_mount_temp.xml for the generated XML")

Waiting for door to close...


2026-01-26 22:08:43,684 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - MethodSet XML saved to: debug_set_mount_temp.xml


✓ Door close completed!
Setting mount temperature to 37°C...
✓ Method started! Request ID: 712229516
Current temperatures:
ODTCSensorValues(timestamp='2026-01-26T23:27:44Z', mount=24.42, mount_monitor=24.27, lid=56.4, lid_monitor=56.81, ambient=22.95, pcb=28.84, heatsink=24.71, heatsink_tec=24.23)
Check debug_set_mount_temp.xml for the generated XML


In [13]:
await tc.read_temperatures()

ODTCSensorValues(timestamp='2026-01-26T23:30:19Z', mount=37.0, mount_monitor=37.03, lid=111.71000000000001, lid_monitor=112.25, ambient=23.52, pcb=26.36, heatsink=23.54, heatsink_tec=23.16)

In [14]:
from pylabrobot.thermocycling.standard import Protocol, Stage, Step

cycle_protocol = Protocol(stages=[
    Stage(
        steps=[
            Step(temperature=[37.0], hold_seconds=10.0),  # 10 second hold at 37°C
            Step(temperature=[60.0], hold_seconds=10.0),    # Ramp to 60°C
            Step(temperature=[10.0], hold_seconds=10.0),    # Ramp down to 10°C
        ],
        repeats=1
    )
])
config = tc.get_default_config(post_heating=True)

print("Waiting for mount to hit target temperature...")
await execution

print("Running temperature cycling protocol...")
execution = await tc.run_protocol(
    protocol=cycle_protocol,
    config=config,
    method_name=None,
    wait=False,  # Non-blocking, returns execution handle
    debug_xml=True,  # Enable XML logging
    xml_output_path="debug_cycling_protocol.xml"  # Save XML to file
)

print(f"✓ Protocol started! Request ID: {execution.request_id}")
if await tc.get_status() == "busy":
    print(f"✓ Method started! Request ID: {execution.request_id}")
    print(f"Current temperatures:\n{await tc.read_temperatures()}")
    print("Check debug_cycling_protocol.xml for the generated XML")


Waiting for mount to hit target temperature...


2026-01-26 22:16:38,830 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - MethodSet XML saved to: debug_cycling_protocol.xml


Running temperature cycling protocol...
✓ Protocol started! Request ID: 710292357
✓ Method started! Request ID: 710292357
Current temperatures:
ODTCSensorValues(timestamp='2026-01-26T23:35:39Z', mount=36.85, mount_monitor=36.96, lid=110.16, lid_monitor=111.19, ambient=23.580000000000002, pcb=25.69, heatsink=23.42, heatsink_tec=23.09)
Check debug_cycling_protocol.xml for the generated XML


# 6. Close the connection

In [15]:
# Wait for completion
await execution

door_opening = await tc.open_door(wait=False)
print(f"Protocol Completed, door opening with request ID: {door_opening.request_id}")
await door_opening
print("✓ Door open completed!")

# Close the connection
await tc.stop()
print("✓ Connection closed.")

Protocol Completed, door opening with request ID: 159742277
✓ Door open completed!
✓ Connection closed.
